In [ ]:
from __future__ import division, print_function

import os
import sys
_path = os.path.abspath("../scripts")
if _path not in sys.path:
    sys.path.insert(0, _path)
from config import galcen_frame, potentials

# Third-party
import astropy.coordinates as coord
import astropy.units as u
import matplotlib.pyplot as pl
import numpy as np
pl.style.use('apw-notebook')
%matplotlib inline
from scipy.interpolate import InterpolatedUnivariateSpline
from scipy.stats import norm
import scipy.optimize as so
import emcee

# Custom
import gary.coordinates as gc
import gary.dynamics as gd
from gary.dynamics import mockstream
from gary.dynamics import orbitfit
import gary.integrate as gi
import gary.potential as gp
from gary.units import galactic

In [ ]:
x = np.logspace(-1,2,256)
xyz = np.zeros((3,x.size))
xyz[0] = x
for k,p in potentials.items():
    pl.loglog(x, p.mass_enclosed(xyz), marker=None, label=k)
pl.legend(loc='lower right')

## Set the potential to run:

In [ ]:
name = 'spherical'
potential = potentials[name]
potential

### Should finish at the same place in both potentials:

In [ ]:
end_c = coord.Galactic(l=125*u.degree, b=31*u.degree, distance=15*u.kpc)
end_v = [2.1*u.mas/u.yr, -0.25*u.mas/u.yr, -100*u.km/u.s]

In [ ]:
end_c_gc = end_c.transform_to(galcen_frame['galactocentric_frame'])
end_v_gc = gc.vhel_to_gal(end_c, end_v[:2], end_v[2], **galcen_frame)
w0 = gd.CartesianPhaseSpacePosition(pos=end_c_gc.represent_as(coord.CartesianRepresentation).xyz,
                                    vel=end_v_gc)

In [ ]:
n_steps = 4000
prog_orbit = potential.integrate_orbit(w0, dt=-1., nsteps=n_steps, t1=float(n_steps),
                                       Integrator=gi.DOPRI853Integrator)
prog_orbit = prog_orbit[::-1]
fig = prog_orbit.plot()
fig.axes[0].plot(end_c_gc.represent_as(coord.CartesianRepresentation).xyz[0],
                 end_c_gc.represent_as(coord.CartesianRepresentation).xyz[1],
                 marker='o')

In [ ]:
stream = mockstream.fardal_stream(potential, prog_orbit=prog_orbit, 
                                  prog_mass=5E5*u.Msun, release_every=1, 
                                  Integrator=gi.DOPRI853Integrator)

In [ ]:
prog_c,prog_v = prog_orbit.to_frame(coord.Galactic, **galcen_frame)
stream_c,stream_v = stream.to_frame(coord.Galactic, **galcen_frame)

In [ ]:
fig,ax = pl.subplots(1,1,figsize=(6,6))
ax.plot(stream_c.l.wrap_at(180*u.degree).degree, stream_c.b.degree, ls='none', alpha=0.25)

In [ ]:
pl.plot(prog_c.l.wrap_at(180*u.deg).degree, prog_c.b.degree, marker=None, alpha=0.25, color='b')
pl.scatter(stream_c.l.wrap_at(180*u.deg).degree, stream_c.b.degree, 
           c=stream_c.distance.value, cmap='Greys_r', s=4, vmin=10, vmax=20)
pl.colorbar()

## Select some "stars" to observe

Only select leading tail and most recently stripped stuff

In [ ]:
idx = np.random.permutation(np.arange(n_steps, stream_c.l.size, dtype=int)[::2])[:64]
obs_c = stream_c[idx]
obs_v = [v[idx] for v in stream_v]

# smaller region of sky
idx = (obs_c.l.degree > 105.) & (obs_c.l.degree < 180.) & (obs_c.b.degree > 30)
obs_c = obs_c[idx]
obs_v = [v[idx] for v in obs_v]

In [ ]:
pl.scatter(obs_c.l.wrap_at(180*u.deg).degree, obs_c.b.degree, s=4, c='k')

---

In [ ]:
np.random.seed(42)
R = orbitfit.compute_stream_rotation_matrix(obs_c, align_lon='max')

In [ ]:
# rotate all data to plot
rot_rep = orbitfit.rotate_sph_coordinate(obs_c, R)

pl.figure(figsize=(10,2))
pl.plot(rot_rep.lon.wrap_at(180*u.deg).degree, rot_rep.lat.degree, ls='none', marker='o', ms=5.)
pl.gca().set_aspect('equal')
pl.xlabel(r'$\phi_1$ [deg]')
pl.ylabel(r'$\phi_2$ [deg]')

### Save data

In [ ]:
import pickle

In [ ]:
for frac_distance_err in [1, 5]:
    np.random.seed(42)

    n_data = len(obs_c)
    data = dict()
    err = dict()

    err['distance'] = frac_distance_err/100. * obs_c.distance
    err['mul'] = 0.1*u.mas/u.yr
    err['mub'] = 0.1*u.mas/u.yr
    err['vr'] = 10.*u.km/u.s
    # err['distance'] = 1E-6*obs_c.distance
    # err['mul'] = 1E-1*u.mas/u.yr
    # err['mub'] = 1E-1*u.mas/u.yr
    # err['vr'] = 1E-6*u.km/u.s

    data['phi1'] = rot_rep.lon
    data['phi2'] = rot_rep.lat
    data['distance'] = obs_c.distance + np.random.normal(0., err['distance'].value, size=n_data)*obs_c.distance.unit
    data['mul'] = obs_v[0] + np.random.normal(0., err['mul'].value, size=n_data)*err['mul'].unit
    data['mub'] = obs_v[1] + np.random.normal(0., err['mub'].value, size=n_data)*err['mub'].unit
    data['vr'] = obs_v[2] + np.random.normal(0., err['vr'].value, size=n_data)*err['vr'].unit
    
    with open("../data/{}_{}percent.pickle".format(name, frac_distance_err), 'wb') as f:
        pickle.dump((data,err), f)